# Introduction
This notebook contains a `Python 2` / `PySpark` script to find the top $N$ words for the positive and negative reviews in a given cluster.

# Notebook Setup

## Initialise modules

In [31]:
import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pymongo
import pandas as pd
import re # Regex for text processing
import os # For setting up Mongo-Spark connector
import csv # To read/write CSV files

## Initialise PySpark session

Load `MongoDB-Spark` connector when starting up `PySpark`.

In [2]:
packages = 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0'

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages {} pyspark-shell'.format(packages)

In [3]:
# Find SPARK_HOME
findspark.init()

# Create SparkSession
spark = (pyspark.sql.SparkSession
         .builder.appName('WordProcessing')
         .getOrCreate())

## Configure Pandas HTML display

In [32]:
pd.set_option('display.max_colwidth', -1)

## Define helper methods

In [4]:
def parse(path):
    '''
    Unzip a json.gz at `path` and returns a generator.
    '''
    g = gzip.open(path, 'rb')
    for line in g:
        yield eval(line)

def import_to_mongo(path, coll, db='hackon', create_index=True):
    '''
    Unzip and import json.gz file from `path` and loads it into mongo server.
    Create database index if `create_index` is True. 
    '''
    # Obtain handle to Mongo database and collection
    client = pymongo.MongoClient()
    collection = client[db][coll]
    
    # Return prematurely if database.collection already exists
    if (collection.count() != 0):
        print '{}.{} already exists on MongoDisk server. Exiting without loading JSON data.'.format(db, coll)
        return
    
    # Insert datapoints into Mongo database
    try:
        collection.insert_many((datapoint for datapoint in parse(path)))
        print 'JSON data successfully imported to Mongo at \'{}.{}.\''.format(db, coll)
    except Exception as e:
        print 'Error loading data.\n{}'.format(e)
        client.close()
        return
    
    if not create_index:
        client.close()
        return
    
    # Create database index for improved searching
    # collection.create_index([('asin', pymongo.ASCENDING), ('reviewerID', pymongo.DESCENDING)])

def load_mongo_to_spark(coll, db='hackon'):
    '''
    Load the Mongo database to a Spark Session and returns the Spark DataFrame
    '''
    try:
        return (spark
                .read
                .format('com.mongodb.spark.sql.DefaultSource')
                .option('uri', 'mongodb://127.0.0.1/{}.{}'.format(db, coll))
                .load())
    except Exception as e:
        print 'Failed to create Spark dataframe.\n{}'.format(e)

def displayDF(sparkDF, n=10):
    '''
    Interactively displays the first n rows of a sparkDF as a pandas dataframe
    '''
    return (sparkDF
            .limit(n)
            .drop('_id', 'unixReviewTime')
            .toPandas())

# Word Processing Script

The function below takes in a Spark DataFrame containing reviews of a particular cluster. It returns a new DataFrame with two appended columns listing the top $N$ words based on *tf-idf* scores for good and bad reviews. 

> By default, *good reviews* are defined as reviews with **4-star** ratings and above, with the rest defined as *bad reviews*.

## Load Mongo toy dataset

In [5]:
def append_toy_clusterID(DF):
    '''
    Append a toy 'clusterID' column to `DF` by assigning each row the clusterID corresponding to its '_id' field's last digit.
    '''
    def _create_toy_clusterID(_id):
        return int(int(_id, 16) % 10)
    
    udf = F.udf(lambda _id: _create_toy_clusterID(_id[0]), T.IntegerType())
    
    return DF.withColumn('clusterID', udf(F.col('_id')))

rawDF = append_toy_clusterID(load_mongo_to_spark('video_games'))

In [6]:
displayDF(rawDF)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,clusterID
0,0700099867,"[8, 12]",1.0,Installing the game was a struggle (because of...,"07 9, 2012",A2HD75EMZR8QLN,123,Pay to unlock content? I don't think so.,6
1,0700099867,"[0, 0]",4.0,If you like rally cars get this game you will ...,"06 30, 2013",A3UR8NLLY1ZHCX,"Alejandro Henao ""Electronic Junky""",Good rally game,7
2,0700099867,"[0, 0]",1.0,1st shipment received a book instead of the ga...,"06 28, 2014",A1INA0F5CWW3J4,"Amazon Shopper ""Mr.Repsol""",Wrong key,8
3,0700099867,"[7, 10]",3.0,"I got this version instead of the PS3 version,...","09 14, 2011",A1DLMTOTHQ4AST,ampgreen,"awesome game, if it did not crash frequently !!",9
4,0700099867,"[2, 2]",4.0,I had Dirt 2 on Xbox 360 and it was an okay ga...,"06 14, 2011",A361M14PU2GUEG,"Angry Ryan ""Ryan A. Forrest""",DIRT 3,0
5,0700099867,"[0, 0]",4.0,"Overall this is a well done racing game, with ...","05 11, 2013",A2UTRVO4FDCBH6,A.R.G.,"Good racing game, terrible Windows Live Requir...",1
6,0700099867,"[11, 13]",5.0,Loved playing Dirt 2 and I thought the graphic...,"08 14, 2011",AN3YYDZAS3O1Y,Bob,A step up from Dirt 2 and that is terrific!,2
7,0700099867,"[1, 4]",1.0,I can't tell you what a piece of dog**** this ...,"11 24, 2012",AQTC623NCESZW,Chesty Puller,Crash 3 is correct name AKA Microsoft,3
8,0700099867,"[0, 1]",4.0,I initially gave this one star because it was ...,"11 14, 2012",A1QJJU33VNC4S7,D@rkFX,A great game ruined by Microsoft's account man...,4
9,0700099867,"[1, 1]",2.0,I still haven't figured this one out. Did ever...,"02 8, 2014",A2JLT2WY0F2HVI,D. Sweetapple,Couldn't get this one to work,5


## Load stopwords
A list of stopwords is loaded as a Python list and broadcasted in PySpark.

In [7]:
# Load stopwords into list
with open('stopwords.csv', 'r') as csvFile:
    fileReader = csv.reader(csvFile)
    stopwords = []
    for word in fileReader:
        stopwords.extend(word)
        
# Add '' to stopwords
stopwords.append('')

# Broadcast stopwords
stopwords_broadcast = spark.sparkContext.broadcast(stopwords)
        
print 'List of stopwords:\n\n{}'.format(stopwords)

List of stopwords:

['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its', 'before', 'with', 'had', 'should', 'to', 'only', 'under', 'ours', 'has', 'do', 'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'did', 'these', 't', 'each', 'where', 'because', 'doing', 'theirs', 'some', 'are', 'our', 'ourselves', 'out', 'what', 'for', 'below', 'does', 'above', 'between', 'she', 'be', 'we', 'after', 'here', 'hers', 'by', 'on', 'about', 'of', 'against', 's', 'or', 'own', 'into', 'yourself', 'down', 'your', 'from', 'her', 'whom', 'there', 'been', 'few', 'too', 'themselves', 'was', 'until', 'more', 'himself', 'that', 'but', 'off', 'herself', 'than', 'those', 'he', 'me', 'myself', 'this', 'up', 'will', 'while', 'can', 'were', 'my', 'and', 'then', 'is', 'in', 'am', 'it', 'an', 'as', 'itself', 'at', 'have', 'further', 'their', 'if', 'again', 'no', 'when', 'same', 'any', 'how', 'other', 'which', 'you', 'who', 'most', 'such', 'why', 'a', 'don', 'i', 'having', 'so', 

## Preprocess `rawDF`

In [8]:
def filter_helpful_reviews(sparkDF, pct_helpful, min_votes):
    '''
    Return a dataframe filtered by comments that are at least `pct_helpful`% helpful 
    and contain a minimum number of votes.
    '''
    def _filter_helpful_reviews(votes, pct_helpful, min_votes):
        '''
        Return bool if comment are at least `pct_helpful`% helpful and have at least `min_votes` vote.
        
        Inputs:
            votes: A list of votes by [helpful, total_votes]
            pct_helpful: A float
            min_votes: A float
        '''
        return (votes[1] >= min_votes and (float(votes[0]) / votes[1] >= pct_helpful))
    
    udf = F.udf(lambda votes: _filter_helpful_reviews(votes, pct_helpful, min_votes), T.BooleanType())
    
    return sparkDF.filter(udf(F.col('helpful')))

def preprocess_DF(rawDF):
    '''
    Preprocess text of dataframe by:
        - selecting relevant columns ('clusterID', 'overall', 'review')
        - concatenating 'summary' and 'reviewText' columns
        - removing punctuations
        - filtering out stopwords
    '''
    
    def _filter_stopwords(text):
        '''
        Input:
            text: A string concatenation of the columns 'summary' and 'reviewText'.
        Returns:
            A list of strings with stopwords filtered out.
        '''
        string_list = re.split(r'\W+', text.lower())
        
        return [word for word in string_list if word not in stopwords_broadcast.value]
    
    udf = F.udf(lambda text: _filter_stopwords(text), T.ArrayType(T.StringType()))
    
    return (rawDF
            .select(F.concat_ws('-', 
                                F.col('asin'), 
                                F.col('reviewerID')).alias('reviewID'),
                    F.col('clusterID'),
                    F.col('overall'), 
                    udf(F.concat_ws(' ', 
                                    F.col('summary'), 
                                    F.col('reviewText'))).alias('reviews'))
            .cache())

In [9]:
preprocessedDF = preprocess_DF(filter_helpful_reviews(rawDF, pct_helpful=.5, min_votes=3))

In [10]:
displayDF(preprocessedDF)

,reviewID,clusterID,overall,reviews
0,0700099867-A2HD75EMZR8QLN,6,1.0,"[pay, unlock, content, think, installing, game..."
1,0700099867-A1DLMTOTHQ4AST,9,3.0,"[awesome, game, crash, frequently, got, versio..."
2,0700099867-AN3YYDZAS3O1Y,2,5.0,"[step, dirt, 2, terrific, loved, playing, dirt..."
3,0700099867-A29KT7UP7DLM1J,5,1.0,"[first, one, much, better, dirt, 2, like, m, b..."
4,0700099867-A18YFCLL3GBD0T,7,1.0,"[abysmal, support, codemasters, would, like, g..."
5,0700099867-A1CB8HH9YJ2YZE,9,3.0,"[fun, pretty, fun, game, buy, cars, tracks, on..."
6,0700099867-A2LQCBLLJVVR5T,0,5.0,"[best, graphics, game, far, must, gamer, codem..."
7,6050036071-A2QQ9KSQ44QZSE,2,5.0,"[yet, another, great, expansion, bought, micro..."
8,6050036071-A91BVZEDZYEWW,4,5.0,"[works, rb3, 360, works, great, another, cool,..."
9,7100027950-A4PPZNQF1X2IY,9,5.0,"[tremendous, game, played, original, zelda, bo..."


## Label ratings for `preprocessedDF`

In [11]:
@F.udf(returnType=T.BooleanType())
def label_review_quality(rating, min_good_review_rating=4):
    '''
    UDF to append a column label of 'isPositiveReview' to the DataFrame.
    
    Inputs:
        rating: Rating from 1.0 to 5.0 (int expressed as float)
    Outputs:
        True if review has a rating of at least `min_good_review_rating`, otherwise False.
    '''
    return True if rating >= min_good_review_rating else False

In [12]:
polarizedDF = (preprocessedDF
               .withColumn('positiveReview', label_review_quality('overall'))
               .cache())
displayDF(polarizedDF)

,reviewID,clusterID,overall,reviews,positiveReview
0,0700099867-A2HD75EMZR8QLN,6,1.0,"[pay, unlock, content, think, installing, game...",False
1,0700099867-A1DLMTOTHQ4AST,9,3.0,"[awesome, game, crash, frequently, got, versio...",False
2,0700099867-AN3YYDZAS3O1Y,2,5.0,"[step, dirt, 2, terrific, loved, playing, dirt...",True
3,0700099867-A29KT7UP7DLM1J,5,1.0,"[first, one, much, better, dirt, 2, like, m, b...",False
4,0700099867-A18YFCLL3GBD0T,7,1.0,"[abysmal, support, codemasters, would, like, g...",False
5,0700099867-A1CB8HH9YJ2YZE,9,3.0,"[fun, pretty, fun, game, buy, cars, tracks, on...",False
6,0700099867-A2LQCBLLJVVR5T,0,5.0,"[best, graphics, game, far, must, gamer, codem...",True
7,6050036071-A2QQ9KSQ44QZSE,2,5.0,"[yet, another, great, expansion, bought, micro...",True
8,6050036071-A91BVZEDZYEWW,4,5.0,"[works, rb3, 360, works, great, another, cool,...",True
9,7100027950-A4PPZNQF1X2IY,9,5.0,"[tremendous, game, played, original, zelda, bo...",True


## Aggregate ratings by `clusterID` and `positiveReview`

In [13]:
@F.udf(returnType=T.ArrayType(T.StringType()))
def flatten(nested_list):
    '''
    Flatten a list of list to a one dimensional list.
    '''
    final_list = []
    for list_ in nested_list:
        final_list.extend(list_)
        
    return final_list

@F.udf(returnType=T.BooleanType())
def bool_invert(boolean):
    '''
    Invert the value of a boolean
    '''
    return not boolean

In [14]:
aggregatedDF = (polarizedDF
                .limit(200)
                .groupBy('clusterID', 'positiveReview')
                .agg(F.collect_list('reviews').alias('collectedReviews'))
                .withColumn('tokens', flatten('collectedReviews'))
                .drop('collectedReviews')
                .sort('clusterID', bool_invert('positiveReview'))
                .cache())
displayDF(aggregatedDF, 100)

,clusterID,positiveReview,tokens
0,0,True,"[best, graphics, game, far, must, gamer, codem..."
1,0,False,"[capcom, thinking, actually, played, saturn, v..."
2,1,True,"[nicely, done, game, good, way, getting, helm,..."
3,1,False,"[good, great, sheer, number, games, bearing, f..."
4,2,True,"[step, dirt, 2, terrific, loved, playing, dirt..."
5,2,False,"[fun, challenging, first, glance, game, may, l..."
6,3,True,"[mega, man, 3d, finally, years, seeing, mega, ..."
7,3,False,"[pretty, awful, know, re, trouble, terrible, v..."
8,4,True,"[works, rb3, 360, works, great, another, cool,..."
9,4,False,"[lack, patience, azure, dreams, isn, really, l..."


## Select top $n$ words ranked by TF-IDF for positive and negative reviews

In [28]:
def top_N_words(DF, N=10):
    '''
    Obtains the top N words of positive and negative reviews respectively, ranked by tf-idf of entries in a DF.
    Inputs:
        DF: A Spark DataFrame containing columns 'clusterID', 'positiveReview' and 'tokens'.
        N:  Number of top-ranking words to keep
    Outputs:
        A DataFrame of schema (_clusterID_, _positiveReview_, top_N_pos, top_N_neg)
    '''
    
    def tf(tokens):
        '''
        Calculate the token frequency (TF) for each review.

        Inputs:
            tokens: A list of token strings.
        Outputs:
            A dictionary of (token, tf).
        '''
        N = len(tokens)

        return {token: float(tokens.count(token)) / N for token in tokens}

    def idf(corpusDF):
        '''
        Calculate the inverse document frequency for the corpusDF.

        Inputs:
            corpusDF: A Spark DataFrame containing columns 'tokens'.
        Outputs:
            A dictionary of (token, idf).
        '''
        # Calculate the number of reviews
        N = corpusDF.count()

        # Create an RDD with entries (uniq_tokens_in_each_review, 1)
        checkpointRDD = (corpusDF
                         .select('tokens')
                         .rdd
                         .flatMap(lambda tokens: list(set(tokens[0])))
                         .map(lambda token: (token, 1)))

        # Produce a dict containing (token, idf)
        return (checkpointRDD
                .reduceByKey(lambda a, b: a + b)
                .mapValues(lambda count: float(count) / N)
                .collectAsMap())
    
    # Calculate tf's as an RDD
    tfRDD = (DF
           .select('clusterID', 'positiveReview', 'tokens')
           .rdd
           .map(lambda (clusterID, positiveReview, tokens): ((clusterID, positiveReview), tf(tokens))))
    
    # Calculate idf's as a dict and return a broadcasted instance
    idfs_broadcast = spark.sparkContext.broadcast(idf(DF.select('tokens')))
    
    # Calculate tfidf as an rdd of (clusterID, positiveReview, top_N_tokens)
    topNwordsRDD = (tfRDD
              .mapValues(lambda tf_dict: [(token, float(tf_dict[token]) / idfs_broadcast.value[token]) for token in tf_dict.keys()])
              .sortBy(lambda ((clusterID, positiveReview), tfidfs): ((clusterID, not positiveReview, sorted(tfidfs, key=lambda (token, tfidf): -tfidf))))
              .mapValues(lambda tfidfs: map(lambda (token, tfidf): token, tfidfs)[:N])
              .map(lambda ((clusterID, positiveReview), top_N_tokens): (clusterID, positiveReview, top_N_tokens))
              .cache())
    
    # Transform topNwordsRDD to an RDD of form (clusterID, top_N_positive, top_N_negative)
    temp_schema = T.StructType([
        (T.StructField('clusterID', T.IntegerType())),
        (T.StructField('positiveReview', T.BooleanType())),
        (T.StructField('topWords', T.ArrayType(T.StringType())))
    ])
    
    finalRDD = (spark
                .createDataFrame(topNwordsRDD, temp_schema)
                .groupBy('clusterID')
                .agg(F.collect_list('topWords').alias('posAndNegWords'))
                .rdd
                .map(lambda (clusterID, words): (clusterID, words[0], words[1])))
    
    # Return finalRDD as a dataframe
    final_schema = T.StructType([
        (T.StructField('clusterID', T.IntegerType())),
        (T.StructField('posWords', T.ArrayType(T.StringType()))),
        (T.StructField('negWords', T.ArrayType(T.StringType())))
    ])
    
    return (spark
            .createDataFrame(finalRDD, final_schema)
            .sort('clusterID')
            .cache())

In [33]:
finalDF = top_N_words(aggregatedDF)
displayDF(finalDF)

,clusterID,posWords,negWords
0,0,"[limited, four, woods, spiders, buddy, granting, worth, minigames, nw0, every]","[things, less, random, fighters, london, ps1, hate, find, access, title]"
1,1,"[limited, multiplayer, managed, vampires, copy, timethis, tombs, four, facilities, code]","[hyper, gold, disables, rob, reptile, meter, discovered, fighters, disc, london]"
2,2,"[stones, multiplayer, managed, consider, pointing, whoever, lack, desirable, souls, yellow]","[renting, atmosphere, ugh, carrie, somewhat, graphics, teenage, years, four, recovering]"
3,3,"[multiplayer, newswoman, forget, depends, copy, graphics, resistance, yellow, remeber, rocketing]","[saying, since, increasingly, skip, particular, years, four, ended, fighters, disc]"
4,4,"[limited, since, customary, angles, pointing, rating, fox, graphics, produce, enjoyable]","[walking, shield, gold, rpgs, swords, random, lee, shape, tools, go]"
5,5,"[default, befriend, consider, personally, unique, walking, four, prices, addictiveness, 9mm]","[since, four, shinryuken, young, oversight, worth, slash, far, continues, soley]"
6,6,"[limited, code, todays, franchise, dance, whoops, personally, dynamic, souls, abit]","[rating, think, stunnning, hands, cheap, anyway, years, buster, dgames, classic]"
7,7,"[managed, whoever, windc, scratch, lack, switches, go, remarkably, launcher2, sega]","[sims, code, played, help, uninstalling, rapture, money, rob, including, go]"
8,8,"[saves, four, immersive, hate, consider, whose, originality, worth, rescue, every]","[limited, battle, poorly, certainly, shack, tm, worth, stocked, risk, worst]"
9,9,"[screenin, concepts, whoever, pros, lack, tombs, four, known, mile, icy]","[concept, chain, incorporate, highways, spiders, go, issues, seemed, devoted, looking]"
